# Rapport 1


## Prétraitement des données 

Nous explorons actuellement les données suivantes :

 - Valeurs des capteurs de :
   - co2
   - humidité
   - luminosité
   - température 
 - Ilots :
   - ilot1
   - ilot2
   - ilot3
   - ouest
   - n57
   - n79

Le prétraitement est le suivant :

 1. Sélection d'un/plusieurs ilot(s) et un/plusieurs capteur(s)
 2. Aggrégation des valeurs par moyenne sur un intervalle
 3. Regroupement des valeurs des différents capteurs correspondant à un même intervalle
 4. Annotation avec la valeur de confort

L'étape 1 est faite en *shell* (cf. [Makefile](../data/Makefile))  
Les étapes 2 et 3 sont réalisées avec *pandas* (cf [preproc.py](../data/preproc.py))  
L'étape 4 est réalisé par le script des M2 (cf. [annotationUnique.py](../data/annotationUnique.py))  

### Format des données en sortie

Exemple : moyenne horaire de l'ilot *ilot2*, tous les capteurs :

| timestamp |co2|hum|lum|temp|confort|
| --- | --- | --- | --- | --- | --- |
|418090|353.2857142857143|41.666666666666664|155.75|24.01714285714286|3.5|
|418091|365.26666666666665|39.78333333333333|116.65|23.885833333333338|3.5|
|418092|347.06666666666666|38.61666666666667|141.31284916201116|23.93316666666666|3.5|
|418093|341.4166666666667|37.733333333333334|152.19553072625698|24.01783333333333|3.5|
|418094|343.85|36.63333333333333|154.29444444444445|24.067000000000007|3.4000000000000004|
|418095|339.68333333333334|35.21666666666667|18.687150837988828|23.906500000000005|3.4000000000000004|
|418096|336.48333333333335|35.7|0.0|23.45416666666666|3.4000000000000004|
|418097|333.3898305084746|36.483333333333334|0.0|23.09372881355932|3.7|
|418098|333.4|38.11666666666667|0.0|22.781500000000005|3.8|
|418099|331.8|40.766666666666666|0.0|22.613000000000007|3.8|
|418100|332.75|42.93333333333333|0.0|22.43266666666666|3.9|
|418101|336.01666666666665|44.45|0.0|22.26566666666667|3.9|
|418102|331.98333333333335|45.8|0.0|22.11183333333333|4.0|
|418103|332.2|46.43333333333333|0.0|21.884666666666664|4.0|
|418104|332.06666666666666|46.45762711864407|0.0|21.699166666666674|4.0|
|418105|328.43333333333334|46.9|0.0|21.515333333333324|4.0|
|418106|327.8333333333333|47.3|0.0|21.459666666666678|4.0|
|418107|387.1525423728813|48.166666666666664|9.759776536312849|21.35186440677966|4.0|
|418108|984.1|50.03333333333333|115.05|23.38216666666667|2.5|



## Valeur de confort 

Le score de confort a deux définitions :

 - Pour un type de capteur donné (ex: co2), le confort $c _ {type}$ est une valeur discrète parmi 0, 1, 2, 3, 4 et 5.
 - Pour un jeu de capteurs complet (co2-hul-lum-temp), le confort $c _ {all}$ est une valeur discrète parmi 0, 0.1, 0.2... jusqu'à 5, soit 51 valeurs possibles (+ erreurs de précision en flottants) : ${c} _ {all} = 0.4 \times c _ {co2} + 0.1 \times c _ {hum} + 0.2 \times c _ {lum} + 0.3 \times c_ {temp}$
 
La seconde définition pose un problème avec les ilots qui ne possèdent pas tous les types de capteurs : actuellement les valeurs manquantes sont considérées égales à 0, ce qui baisse arbitrairement le score.
Les scores restent *proportionnels* si les valeurs manquantes sont toujours les mêmes, mais perdrent leur sens sinon.

Peut-être peut-on les prédire et utiliser pour les valeurs manquantes les *valeurs les plus probablement associées* aux valeurs présentes ?

In [ ]:
# Import
import pandas
import numpy

# Script des M1 (fonction de confort)
import lib.annotationUnique as anno


# Constantes
delimiter = ';'
interval = 3600
stypes = ['co2','hum','lum','temp']

# Fichiers
in_file = 'data/sample.ilot2.all.csv'
out_file = 'data/sample.ilot2.all.%d.csv' % interval



# Fonction d'annotation d'une ligne
def annotate(row):
    """Annotation avec la fonction ad-hoc (un seul type) ou la fonction anntationUnique (plusieurs types, valeurs absentes à zéro)"""
      
    # Si un seul type dans l'index (+confort -> len==2): fonction ad-hoc
    if len(row.index) == 2:
        if (row.index[0] == 'co2'):
            row['confort'] = anno.co2(row['co2'])
        elif (row.index[0] == 'hum'):
            row['confort'] = anno.humidite(row['hum'])
        elif (row.index[0] == 'lum'):
            row['confort'] = anno.luminosite(row['lum'])
        elif (row.index[0] == 'temp'):
            row['confort'] = anno.temperature(row['temp'])
        else:
            print("index '{}' not recognized".format(index[0]),
                  file=sys.stderr)

            
    # Si plusieurs types : annotationCapteur avec 0 pour valeurs manquantes
    else:
        getvalue = lambda t: row[t] if t in row.index else 0
        row['confort'] = anno.annotationCapteur(getvalue('temp'),
                                                getvalue('lum'),
                                                getvalue('co2'),
                                                getvalue('hum'))
    return row





# Lire le fichier de données
data = pandas.read_csv(in_file, sep=delimiter)

# Agréger par moyenne sur l'intervalle (3600 secondes)
data['timestamp'] = data['timestamp'] // 3600
avg = data.groupby(['timestamp', 'type']).mean()

# Préparer l'output
output = pandas.DataFrame(index=numpy.unique(data['timestamp']),
                          columns=stypes+["confort"])

# Recopier les valeurs
for (ts,stype), row in avg.iterrows():
    output.at[ts,stype] = row[0]

# Annoter avec le score de confort
output = output.apply(annotate, axis=1)

# Et voilà !
output.to_csv(out_file, sep=delimiter)

In [ ]:
# Affichage du fichier produit
with open(out_file, 'r') as f:
    for line in f:
        print(line, end="")